# Make Actual Story Vectors
## 12/7/20
## Make a dict, where for each story there is the wordvec made from the actual stories read 

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
from random import randrange
from sklearn.metrics import jaccard_score
import random

from sklearn.decomposition import PCA #for cluster analysis
from gensim.models import KeyedVectors #for word embeddings
import matplotlib.pyplot as plt #for plotting
import os #for importing
import pickle #for loading transcripts
from scipy.stats import pearsonr 

# from _DRAFT_20200604_functions import * #includes constants and score function
from tqdm import tqdm_notebook #for progress bar

%autosave 5

Autosaving every 5 seconds


# I. Imports
## A. Read story texts

In [2]:

files1 = glob.glob('/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/*')
len(files1)

16

In [3]:
for filenam in files1: 
    print(filenam)

/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/32
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/34
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/33
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/11
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/42
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/21
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/44
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/43
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/31
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/41
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/24
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/story_texts_ns/23
/Users/alexreblando/Documents/GitHub/data_analysis_ebs/fmri/stor

In [4]:
files = {}
for filenam in files1: 
    with open(filenam, "r") as file: 
        if filenam in files: 
            continue 
        else:
            x = filenam.split('ns/')[1]
            files[x] = file.read() 

## B. Importing wiki vectors

In [5]:
wikipath = 'rolando/wiki-news-300d-1M.vec'
wv_model = KeyedVectors.load_word2vec_format(wikipath)

# II. Functions

In [7]:
wv_dim = 300

# FastText preprocessing, based on bittlingmayer/ft_wiki_preproc.py
# Remove special characters, put spaces between all tokens
SUB = ["s/’/'/g", "s/′/'/g", "s/''/ /g", "s/'/ ' /g", 's/“/"/g', 's/”/"/g', 's/"/ /g', "s/\\./ \\. /g", "s/<br \\/>/ /g", "s/, / , /g", "s/(/ ( /g", "s/)/ ) /g", "s/\\!/ \\! /g", "s/\\?/ \\? /g", "s/\\;/ /g", "s/\\:/ /g", "s/-/ - /g", "s/=/ /g", "s/=/ /g", "s/*/ /g", "s/|/ /g", "s/«/ /g", 
       "s/…/ /g", "s/‘/ /g", "s/í/ /g", "s/ñ/ /g", "s/\x84/ /g", "s/î/ /g", "s/ó/ /g", "s/\x83/ /g", "s/ï/ /g", "s/õ/ /g",
       "s/ò/ /g", "s/,/ /g", "s/ô/ /g", "s/\x92/ /g", "s/é/ /g", "s/\x8e/ /g", "s/â\x80¦/ /g", "s/\x91/ /g", "s/\x93/ /g",
       "s/\x94/ /g", "s/ã®/ /g", "s/ã¨/ /g", "s/ã©/ /g",
       "s/\â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x9d/ /g", "s/â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x98/ /g",
       "s/â/ /g"]

def __normalize_text(s):
    for sg in SUB:
        rep = sg.replace('\\','').split('/')
        s = s.replace(rep[1], rep[2])
    s = s.replace('/',' ')
    return s

def __spaces(s):
    return ' '.join(s.split())

def __digits(s):
    return ''.join(filter(lambda c: not c.isdigit(), s))

# def preproc(s):
#     return __punctuation(__spaces(__digits(__normalize_text(s.lower()))))

def preproc(s):
    return (__spaces(__digits(__normalize_text(s.lower()))))

def __punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

def word2vecSent(sentence, model = 'fasttext'):
    wv_dim = 300 #for glove and fasttext
    
    if model == 'glove':
        wvmodel = glove_model
    elif model == 'fasttext':
        wvmodel = wv_model
        
    words = preproc(sentence).split(' ')
    wv = np.zeros((len(words), wv_dim))
    for i in range(len(words)):
        if words[i] in wvmodel.vocab:
            wv[i,:] = wvmodel.word_vec(words[i])
    
    return wv, words

## III. Making the actual story vectors
### A. Getting Word Vectors for each free recall 
#### dict{story: story_text}

In [15]:
wv_matrices = {}
gran_stories = {}

# Iterating through each key
for file in files:
    these_wvs, these_words = word2vecSent(files[file])
    # Story template
    wv_matrices[file] = np.mean(these_wvs, axis = 0)
    # Matrix with vectors of individual words, and list with words
    gran_stories[file] = {'wvs': these_wvs, 'words': these_words}

### C. Export Word Vector Matrices and sums

In [34]:
pickle.dump( wv_matrices, open( 'actual_story_vectors', "wb" ) )

## D. Export gran_stories

In [19]:
pickle.dump( gran_stories, open( 'gran_stories', "wb" ) )